In [1]:
import os
import json
from pathlib import Path
import zipfile
import email
from email.policy import default
from email.parser import Parser
from datetime import timezone
from collections import namedtuple

import pandas as pd
import s3fs
from bs4 import BeautifulSoup
from dateutil.parser import parse
from chardet.universaldetector import UniversalDetector

from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.pipeline import Transformer
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StringType

import pandas as pd

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)
data_dir = current_dir.joinpath('data')
data_dir.mkdir(parents=True, exist_ok=True)
enron_data_dir = data_dir.joinpath('enron')

output_columns = [
        'payload',
        'text',
        'Message_ID',
        'Date',
        'From',
        'To',
        'Subject',
        'Mime-Version',
        'Content-Type',
        'Content-Transfer-Encoding',
        'X-From',
        'X-To',
        'X-cc',
        'X-bcc',
        'X-Folder',
        'X-Origin',
        'X-FileName',
        'Cc',
        'Bcc' 
]

columns = [column.replace('-', '_') for column in output_columns]

ParsedEmail = namedtuple('ParsedEmail', columns)

spark = SparkSession\
    .builder\
    .appName("Assignment04")\
    .getOrCreate()

In [2]:
# print(msg.get_payload())

In [81]:
for i in msg:
    print(i)

Message-ID
Date
From
To
Subject
Mime-Version
Content-Type
Content-Transfer-Encoding
X-From
X-To
X-cc
X-bcc
X-Folder
X-Origin
X-FileName


In [2]:
spark

The following code loads data to your local JupyterHub instance. You only need to run this once. 

In [3]:
# def copy_data_to_local():
#     dst_data_path = data_dir.joinpath('enron.zip')
#     endpoint_url='https://storage.budsc.midwest-datascience.com'
#     enron_data_path = 'data/external/enron.zip'

#     s3 = s3fs.S3FileSystem(
#         anon=True,
#         client_kwargs={
#             'endpoint_url': endpoint_url
#         }
#     )

    
#     s3.get(enron_data_path, str(dst_data_path))
    
#     with zipfile.ZipFile(dst_data_path) as f_zip:
#         f_zip.extractall(path=data_dir)
    
# copy_data_to_local()

Copy the data from one folder to another:

In [4]:
# ! cp -a /home/jovyan/dsc650/data/external/enron/. /home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron

This code reads emails and creates a Spark dataframe with three columns. 

## Assignment 4.1

In [3]:
def read_raw_email(email_path):
    detector = UniversalDetector()
    
    try:
        with open(email_path) as f:
            original_msg = f.read()
    except UnicodeDecodeError:
        detector.reset()
        with open(email_path, 'rb') as f:
            for line in f.readlines():
                detector.feed(line)
                if detector.done:
                    break
        detector.close()
        encoding = detector.result['encoding']
        with open(email_path, encoding=encoding) as f:
            original_msg = f.read()
            
    return original_msg 

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.pipeline import Transformer
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StringType

In [4]:
enron_data_dir

PosixPath('/home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron')

In [6]:
# Path(root)

In [7]:
def make_spark_df():
    records = []
    n = 0
    for root, dirs, files in os.walk(enron_data_dir):
#         print(f'root:  {root}\n {type(root)}')
#         test = root.rsplit('/',2)
#         print(f'TEST: {test}')
#         print(f'TEST2: {test[-2]}')
        
#         if dirs == []:
#             pass
#         else:
#             print(f'dirs:  {dirs}\n {type(dirs)}')
#         if files == []:
#             pass
#         else:
#             print(f'files: {files}\n {type(files)}\n') 
        
        for file_path in files:
            test = root.rsplit('/',2)
#             print(test)
#             print(f'TEST3: {test[-2]}')
            record = {}
#             print(f'\nfilepath: {file_path}\n {type(file_path)}\n')

#             print(f'root:  {root}\n {type(root)}')
#             print(f'Path(root): {Path(root)}\n')
#             print(f'file_path: {file_path}\n')
            
            current_path = Path(root).joinpath(file_path)
#             print(f'current path: {current_path}\n')
            ## Current path is now the file path to the current email.
            
            
            ## Use this path to read the following information:
            
            ## id (The relative path of the email message)
#             id_path = current_path
            id_path = str(current_path)
            record['id'] = id_path
#             print(f'\n RECORD1: {record}')
            
            ## username (Hint: It is the root folder)
            username = root.rsplit('/', 2)[-2]
            record['username'] = username
#             print(f'\n RECORD2: {record}')
            
            ## original_msg
            
            msg = read_raw_email(current_path)
#             print(msg)
            record['original_msg'] = msg
            print(f'\n RECORD3: {record}\n')
            
            records.append(record)
#             print(records)
            n +=1
            if n > 5:
                return records
#         print(records)
    return records
        
    ## TODO: Complete the code to code to create the Spark dataframe
#     schemaString = "id_path username original_msg"
    
#     return spark.createDataFrame()

In [107]:
enron_data_dir

PosixPath('/home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron')

In [108]:
records = []
for root, dirs, files in os.walk(enron_data_dir):
    for file_path in files:
        record = {}
        current_path = str(Path(root).joinpath(file_path))

In [109]:
current_path

'/home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron/reitmeyer-j/inbox/453_'

In [113]:
orig_msg = read_raw_email(current_path)

In [135]:
print(orig_msg)

Message-ID: <20636892.1075861431702.JavaMail.evans@thyme>
Date: Mon, 19 Nov 2001 05:14:25 -0800 (PST)
From: ina.rangel@enron.com
To: jay.reitmeyer@enron.com, m..scott@enron.com, mog.heu@enron.com, 
	frank.ermis@enron.com, p..south@enron.com, ina.rangel@enron.com, 
	m..tholt@enron.com, jason.huang@enron.com, matthew.lenhart@enron.com, 
	mike.grigsby@enron.com, tori.kuykendall@enron.com, 
	jason.wolfe@enron.com, matt.smith@enron.com, keith.holst@enron.com, 
	k..allen@enron.com
Subject: FW: STENO NUMBERS
Cc: laura.vuittonet@enron.com, jessica.presas@enron.com, 
	kimberly.bates@enron.com, alex.villarreal@enron.com, 
	daniel.quezada@enron.com
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
Bcc: laura.vuittonet@enron.com, jessica.presas@enron.com, 
	kimberly.bates@enron.com, alex.villarreal@enron.com, 
	daniel.quezada@enron.com
X-From: Rangel, Ina </O=ENRON/OU=NA/CN=RECIPIENTS/CN=IRANGEL>
X-To: Reitmeyer, Jay </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Jreit

In [114]:
orig_msg

'Message-ID: <20636892.1075861431702.JavaMail.evans@thyme>\nDate: Mon, 19 Nov 2001 05:14:25 -0800 (PST)\nFrom: ina.rangel@enron.com\nTo: jay.reitmeyer@enron.com, m..scott@enron.com, mog.heu@enron.com, \n\tfrank.ermis@enron.com, p..south@enron.com, ina.rangel@enron.com, \n\tm..tholt@enron.com, jason.huang@enron.com, matthew.lenhart@enron.com, \n\tmike.grigsby@enron.com, tori.kuykendall@enron.com, \n\tjason.wolfe@enron.com, matt.smith@enron.com, keith.holst@enron.com, \n\tk..allen@enron.com\nSubject: FW: STENO NUMBERS\nCc: laura.vuittonet@enron.com, jessica.presas@enron.com, \n\tkimberly.bates@enron.com, alex.villarreal@enron.com, \n\tdaniel.quezada@enron.com\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nBcc: laura.vuittonet@enron.com, jessica.presas@enron.com, \n\tkimberly.bates@enron.com, alex.villarreal@enron.com, \n\tdaniel.quezada@enron.com\nX-From: Rangel, Ina </O=ENRON/OU=NA/CN=RECIPIENTS/CN=IRANGEL>\nX-To: Reitmeyer, Jay </O=ENRO

In [116]:
testParsed = parse_email(orig_msg)

In [119]:
print(testParsed)

ParsedEmail(payload='None', text='\nHere is a list for the schedulers downstairs...we will get you a more completed one later.\n -----Original Message-----\nFrom: \tSaldana, Alex  \nSent:\tFriday, November 16, 2001 3:54 PM\nTo:\tRangel, Ina\nSubject:\tSTENO NUMBERS\n\n\nINA,\n\nTHIS IS WHAT WE HAVE FOR NOW.  MONDAY EVERYONE THAT DOES NOT HAVE A STENO WILL.  IF YOU NEED THE LIST OF NEW STENOS PLEASE  CONTACT BRANDEE JACKSON BECAUSE I WILL BE OUT ON VACATION.\n\nTHANKS.  SORRY IT TOOK SO LONG.\n\n \nAlex Saldana \nAdministrative Assistant\nEnron Networks \n(713) 345-7389\n\n', Message_ID='None', Date='Mon, 19 Nov 2001 05:14:25 -0800', From='ina.rangel@enron.com', To='jay.reitmeyer@enron.com, m..scott@enron.com, mog.heu@enron.com, frank.ermis@enron.com, p..south@enron.com, ina.rangel@enron.com, m..tholt@enron.com, jason.huang@enron.com, matthew.lenhart@enron.com, mike.grigsby@enron.com, tori.kuykendall@enron.com, jason.wolfe@enron.com, matt.smith@enron.com, keith.holst@enron.com, k..allen

In [121]:
testParsed.text

'\nHere is a list for the schedulers downstairs...we will get you a more completed one later.\n -----Original Message-----\nFrom: \tSaldana, Alex  \nSent:\tFriday, November 16, 2001 3:54 PM\nTo:\tRangel, Ina\nSubject:\tSTENO NUMBERS\n\n\nINA,\n\nTHIS IS WHAT WE HAVE FOR NOW.  MONDAY EVERYONE THAT DOES NOT HAVE A STENO WILL.  IF YOU NEED THE LIST OF NEW STENOS PLEASE  CONTACT BRANDEE JACKSON BECAUSE I WILL BE OUT ON VACATION.\n\nTHANKS.  SORRY IT TOOK SO LONG.\n\n \nAlex Saldana \nAdministrative Assistant\nEnron Networks \n(713) 345-7389\n\n'

In [131]:
parse_email(orig_msg).text

'\nHere is a list for the schedulers downstairs...we will get you a more completed one later.\n -----Original Message-----\nFrom: \tSaldana, Alex  \nSent:\tFriday, November 16, 2001 3:54 PM\nTo:\tRangel, Ina\nSubject:\tSTENO NUMBERS\n\n\nINA,\n\nTHIS IS WHAT WE HAVE FOR NOW.  MONDAY EVERYONE THAT DOES NOT HAVE A STENO WILL.  IF YOU NEED THE LIST OF NEW STENOS PLEASE  CONTACT BRANDEE JACKSON BECAUSE I WILL BE OUT ON VACATION.\n\nTHANKS.  SORRY IT TOOK SO LONG.\n\n \nAlex Saldana \nAdministrative Assistant\nEnron Networks \n(713) 345-7389\n\n'

In [134]:
testParsed

'None'

In [123]:
print(testParsed.text)


Here is a list for the schedulers downstairs...we will get you a more completed one later.
 -----Original Message-----
From: 	Saldana, Alex  
Sent:	Friday, November 16, 2001 3:54 PM
To:	Rangel, Ina
Subject:	STENO NUMBERS


INA,

THIS IS WHAT WE HAVE FOR NOW.  MONDAY EVERYONE THAT DOES NOT HAVE A STENO WILL.  IF YOU NEED THE LIST OF NEW STENOS PLEASE  CONTACT BRANDEE JACKSON BECAUSE I WILL BE OUT ON VACATION.

THANKS.  SORRY IT TOOK SO LONG.

 
Alex Saldana 
Administrative Assistant
Enron Networks 
(713) 345-7389




In [120]:
len(testParsed)

19

In [117]:
for i in testParsed:
    print(i)

None

Here is a list for the schedulers downstairs...we will get you a more completed one later.
 -----Original Message-----
From: 	Saldana, Alex  
Sent:	Friday, November 16, 2001 3:54 PM
To:	Rangel, Ina
Subject:	STENO NUMBERS


INA,

THIS IS WHAT WE HAVE FOR NOW.  MONDAY EVERYONE THAT DOES NOT HAVE A STENO WILL.  IF YOU NEED THE LIST OF NEW STENOS PLEASE  CONTACT BRANDEE JACKSON BECAUSE I WILL BE OUT ON VACATION.

THANKS.  SORRY IT TOOK SO LONG.

 
Alex Saldana 
Administrative Assistant
Enron Networks 
(713) 345-7389


None
Mon, 19 Nov 2001 05:14:25 -0800
ina.rangel@enron.com
jay.reitmeyer@enron.com, m..scott@enron.com, mog.heu@enron.com, frank.ermis@enron.com, p..south@enron.com, ina.rangel@enron.com, m..tholt@enron.com, jason.huang@enron.com, matthew.lenhart@enron.com, mike.grigsby@enron.com, tori.kuykendall@enron.com, jason.wolfe@enron.com, matt.smith@enron.com, keith.holst@enron.com, k..allen@enron.com
FW: STENO NUMBERS
None
None
None
None
None
None
None
None
None
None
laura.vuitt

In [8]:
records = make_spark_df()


 RECORD3: {'id': '/home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron/gay-r/to_do/1_', 'username': 'gay-r', 'original_msg': 'Message-ID: <24144526.1075855390413.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:23:00 -0700 (PDT)\nFrom: randall.gay@enron.com\nSubject: Do Not Delete - Repeat parent for EP Bondad reduced 85.000\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Randall L Gay\nX-To: \nX-cc: \nX-bcc: \nX-Folder: \\Randall_Gay_Jan2002_1\\Gay, Randall L.\\To Do\nX-Origin: Gay-R\nX-FileName: rgay (Non-Privileged).pst\n\n\tTASK ASSIGNMENT\n\n\nStatus:\t\tcompleted\n\nTask Priority:\t\t\nTask Due On:\t\t8/4/2000\nTask Start Date:\t11/9/1998'}


 RECORD3: {'id': '/home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron/gay-r/to_do/12_', 'username': 'gay-r', 'original_msg': 'Message-ID: <23002226.1075855390645.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:23:00 -0700 (PDT)\nFrom: randall.gay@enron.com\nSubje

In [9]:
records

[{'id': '/home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron/gay-r/to_do/1_',
  'username': 'gay-r',
  'original_msg': 'Message-ID: <24144526.1075855390413.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:23:00 -0700 (PDT)\nFrom: randall.gay@enron.com\nSubject: Do Not Delete - Repeat parent for EP Bondad reduced 85.000\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Randall L Gay\nX-To: \nX-cc: \nX-bcc: \nX-Folder: \\Randall_Gay_Jan2002_1\\Gay, Randall L.\\To Do\nX-Origin: Gay-R\nX-FileName: rgay (Non-Privileged).pst\n\n\tTASK ASSIGNMENT\n\n\nStatus:\t\tcompleted\n\nTask Priority:\t\t\nTask Due On:\t\t8/4/2000\nTask Start Date:\t11/9/1998'},
 {'id': '/home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron/gay-r/to_do/12_',
  'username': 'gay-r',
  'original_msg': 'Message-ID: <23002226.1075855390645.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:23:00 -0700 (PDT)\nFrom: randall.gay@enron.com\nSubject: Do Not D

In [10]:
# pPath = records[0]['id']
# pPath

In [96]:
# str(pPath)

In [11]:
schemaStr = "id username original_msg"

In [12]:
from pyspark.sql.types import StructField
fields = [StructField(field_name, StringType(), True) for field_name in schemaStr.split()]

In [13]:
fields

[StructField(id,StringType,true),
 StructField(username,StringType,true),
 StructField(original_msg,StringType,true)]

In [14]:
from pyspark.sql.types import StructType
schema = StructType(fields)

In [15]:
schema

StructType(List(StructField(id,StringType,true),StructField(username,StringType,true),StructField(original_msg,StringType,true)))

In [16]:
dfEmails = spark.createDataFrame(records, schema)

In [17]:
dfEmails

DataFrame[id: string, username: string, original_msg: string]

In [18]:
dfEmails.show()

+--------------------+--------+--------------------+
|                  id|username|        original_msg|
+--------------------+--------+--------------------+
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2414...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2300...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2677...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <5586...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2400...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <1780...|
+--------------------+--------+--------------------+



In [105]:
dfEmails.printSchema()

root
 |-- id: string (nullable = true)
 |-- username: string (nullable = true)
 |-- original_msg: string (nullable = true)



In [37]:
from pyspark.sql.types import StructField, StructType

def make_spark_df():
    records = []
#     n = 0
    for root, dirs, files in os.walk(enron_data_dir):
        
        for file_path in files:

            record = {}

            current_path = Path(root).joinpath(file_path)

            ## Current path is now the file path to the current email.
                        
            ## Use this path to read the following information:
            
            ## id (The relative path of the email message)
            id_path = str(current_path)
            record['id'] = id_path
            
            ## username (Hint: It is the root folder)
            username = root.rsplit('/', 2)[-2]
            record['username'] = username
            
            ## original_msg
            msg = read_raw_email(current_path)
            record['original_msg'] = msg
            
            records.append(record)
#             n +=1
#             if n > 5:
# #                 return records
#                 pass
#     return records
        
    ## TODO: Complete the code to code to create the Spark dataframe
    # Encode schema in a string
    schemaString = "id_path username original_msg"
    fields = [StructField(field_name, StringType(), True) for field_name in schemaStr.split()]
    schema = StructType(fields)
    # Apply the schema
    return spark.createDataFrame(records, schema)

In [35]:
del(dfEmails)

In [38]:
dfEmails = make_spark_df()

In [39]:
dfEmails

DataFrame[id: string, username: string, original_msg: string]

In [40]:
dfEmails.show()

+--------------------+--------+--------------------+
|                  id|username|        original_msg|
+--------------------+--------+--------------------+
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2414...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2300...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2677...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <5586...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2400...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <1780...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <1399...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <1311...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2330...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2590...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2495...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2526...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <1070...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2709...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <2319...|
|/home/jovyan/dsc6...|   gay-r|Message-ID: <24

In [41]:
dfEmails.printSchema()

root
 |-- id: string (nullable = true)
 |-- username: string (nullable = true)
 |-- original_msg: string (nullable = true)



In [65]:
from pyspark.sql.types import StructField, StructType

def read_raw_email(email_path):
    detector = UniversalDetector()
    
    try:
        with open(email_path) as f:
            original_msg = f.read()
    except UnicodeDecodeError:
        detector.reset()
        with open(email_path, 'rb') as f:
            for line in f.readlines():
                detector.feed(line)
                if detector.done:
                    break
        detector.close()
        encoding = detector.result['encoding']
        with open(email_path, encoding=encoding) as f:
            original_msg = f.read()
            
    return original_msg 

def make_spark_df():
    records = []
    for root, dirs, files in os.walk(enron_data_dir):
        for file_path in files:
            record = {}
            current_path = str(Path(root).joinpath(file_path))
#             print(file_path)
#             print(current_path)
            
            ## id (The relative path of the email message)
            record['id_path'] = current_path
            record['id'] = current_path.split('/', 9)[-1]
            print(record['id'])
            ## username (Hint: It is the root folder)
            record['username'] = root.rsplit('/', 2)[-2]
            ## original_msg
            record['original_msg'] = read_raw_email(current_path)
            
#             print(record)
            # Append record to list
            records.append(record)
            
    ## TODO: Complete the code to code to create the Spark dataframe
    # Encode schema in a string
    schemaString = "id username original_msg"
    fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
    schema = StructType(fields)
    # Apply the schema    
    return spark.createDataFrame(records, schema)

In [66]:
df = make_spark_df()

gay-r/to_do/1_
gay-r/to_do/12_
gay-r/to_do/6_
gay-r/to_do/5_
gay-r/to_do/9_
gay-r/to_do/13_
gay-r/to_do/7_
gay-r/to_do/11_
gay-r/to_do/2_
gay-r/to_do/4_
gay-r/deleted_items/1_
gay-r/deleted_items/25_
gay-r/deleted_items/87_
gay-r/deleted_items/35_
gay-r/deleted_items/107_
gay-r/deleted_items/68_
gay-r/deleted_items/60_
gay-r/deleted_items/6_
gay-r/deleted_items/56_
gay-r/deleted_items/85_
gay-r/deleted_items/79_
gay-r/deleted_items/26_
gay-r/deleted_items/45_
gay-r/deleted_items/10_
gay-r/deleted_items/43_
gay-r/deleted_items/98_
gay-r/deleted_items/96_
gay-r/deleted_items/67_
gay-r/deleted_items/48_
gay-r/deleted_items/44_
gay-r/deleted_items/105_
gay-r/deleted_items/39_
gay-r/deleted_items/50_
gay-r/deleted_items/100_
gay-r/deleted_items/90_
gay-r/deleted_items/61_
gay-r/deleted_items/9_
gay-r/deleted_items/80_
gay-r/deleted_items/29_
gay-r/deleted_items/78_
gay-r/deleted_items/58_
gay-r/deleted_items/36_
gay-r/deleted_items/89_
gay-r/deleted_items/47_
gay-r/deleted_items/54_
gay-r/d

gay-r/all_documents/253_
gay-r/all_documents/42_
gay-r/all_documents/358_
gay-r/all_documents/418_
gay-r/all_documents/279_
gay-r/all_documents/293_
gay-r/all_documents/84_
gay-r/all_documents/378_
gay-r/all_documents/290_
gay-r/all_documents/222_
gay-r/all_documents/383_
gay-r/all_documents/205_
gay-r/all_documents/73_
gay-r/all_documents/251_
gay-r/all_documents/110_
gay-r/all_documents/317_
gay-r/all_documents/127_
gay-r/all_documents/226_
gay-r/all_documents/21_
gay-r/all_documents/284_
gay-r/all_documents/262_
gay-r/notes_inbox/1_
gay-r/notes_inbox/25_
gay-r/notes_inbox/12_
gay-r/notes_inbox/6_
gay-r/notes_inbox/26_
gay-r/notes_inbox/10_
gay-r/notes_inbox/5_
gay-r/notes_inbox/9_
gay-r/notes_inbox/13_
gay-r/notes_inbox/20_
gay-r/notes_inbox/18_
gay-r/notes_inbox/23_
gay-r/notes_inbox/15_
gay-r/notes_inbox/7_
gay-r/notes_inbox/17_
gay-r/notes_inbox/11_
gay-r/notes_inbox/2_
gay-r/notes_inbox/14_
gay-r/notes_inbox/24_
gay-r/notes_inbox/16_
gay-r/notes_inbox/8_
gay-r/notes_inbox/27_
ga

gay-r/discussion_threads/16_
gay-r/discussion_threads/199_
gay-r/discussion_threads/207_
gay-r/discussion_threads/55_
gay-r/discussion_threads/31_
gay-r/discussion_threads/225_
gay-r/discussion_threads/146_
gay-r/discussion_threads/101_
gay-r/discussion_threads/120_
gay-r/discussion_threads/235_
gay-r/discussion_threads/99_
gay-r/discussion_threads/57_
gay-r/discussion_threads/95_
gay-r/discussion_threads/8_
gay-r/discussion_threads/63_
gay-r/discussion_threads/59_
gay-r/discussion_threads/27_
gay-r/discussion_threads/53_
gay-r/discussion_threads/124_
gay-r/discussion_threads/182_
gay-r/discussion_threads/197_
gay-r/discussion_threads/69_
gay-r/discussion_threads/64_
gay-r/discussion_threads/62_
gay-r/discussion_threads/188_
gay-r/discussion_threads/221_
gay-r/discussion_threads/172_
gay-r/discussion_threads/93_
gay-r/discussion_threads/66_
gay-r/discussion_threads/183_
gay-r/discussion_threads/71_
gay-r/discussion_threads/37_
gay-r/discussion_threads/133_
gay-r/discussion_threads/125_

davis-d/all_documents/287_
davis-d/all_documents/258_
davis-d/all_documents/174_
davis-d/all_documents/569_
davis-d/all_documents/405_
davis-d/all_documents/252_
davis-d/all_documents/304_
davis-d/all_documents/312_
davis-d/all_documents/308_
davis-d/all_documents/468_
davis-d/all_documents/87_
davis-d/all_documents/278_
davis-d/all_documents/119_
davis-d/all_documents/469_
davis-d/all_documents/318_
davis-d/all_documents/232_
davis-d/all_documents/35_
davis-d/all_documents/154_
davis-d/all_documents/277_
davis-d/all_documents/234_
davis-d/all_documents/143_
davis-d/all_documents/121_
davis-d/all_documents/107_
davis-d/all_documents/12_
davis-d/all_documents/385_
davis-d/all_documents/68_
davis-d/all_documents/180_
davis-d/all_documents/402_
davis-d/all_documents/81_
davis-d/all_documents/449_
davis-d/all_documents/369_
davis-d/all_documents/165_
davis-d/all_documents/393_
davis-d/all_documents/259_
davis-d/all_documents/113_
davis-d/all_documents/173_
davis-d/all_documents/313_
davis-

davis-d/save_the_date/3_
davis-d/save_the_date/28_
davis-d/save_the_date/38_
davis-d/save_the_date/19_
davis-d/save_the_date/42_
davis-d/save_the_date/21_
davis-d/accting/1_
davis-d/accting/2_
davis-d/accting/3_
davis-d/fyi/1_
davis-d/fyi/25_
davis-d/fyi/12_
davis-d/fyi/6_
davis-d/fyi/26_
davis-d/fyi/10_
davis-d/fyi/5_
davis-d/fyi/9_
davis-d/fyi/13_
davis-d/fyi/20_
davis-d/fyi/18_
davis-d/fyi/29_
davis-d/fyi/23_
davis-d/fyi/15_
davis-d/fyi/7_
davis-d/fyi/17_
davis-d/fyi/11_
davis-d/fyi/32_
davis-d/fyi/2_
davis-d/fyi/30_
davis-d/fyi/14_
davis-d/fyi/24_
davis-d/fyi/16_
davis-d/fyi/31_
davis-d/fyi/8_
davis-d/fyi/27_
davis-d/fyi/4_
davis-d/fyi/22_
davis-d/fyi/3_
davis-d/fyi/28_
davis-d/fyi/19_
davis-d/fyi/21_
davis-d/vargas_becton_lamb_galvan/1_
davis-d/vargas_becton_lamb_galvan/2_
davis-d/vargas_becton_lamb_galvan/3_
davis-d/sent_items/1_
davis-d/sent_items/25_
davis-d/sent_items/35_
davis-d/sent_items/12_
davis-d/sent_items/6_
davis-d/sent_items/26_
davis-d/sent_items/45_
davis-d/sent_it

...

zipper-a/deleted_items/34_
zipper-a/deleted_items/74_
zipper-a/deleted_items/356_
zipper-a/deleted_items/351_
zipper-a/deleted_items/111_
zipper-a/deleted_items/231_
zipper-a/deleted_items/556_
zipper-a/deleted_items/257_
zipper-a/deleted_items/523_
zipper-a/deleted_items/504_
zipper-a/deleted_items/336_
zipper-a/deleted_items/247_
zipper-a/deleted_items/377_
zipper-a/deleted_items/539_
zipper-a/deleted_items/108_
zipper-a/deleted_items/566_
zipper-a/deleted_items/359_
zipper-a/deleted_items/431_
zipper-a/deleted_items/145_
zipper-a/deleted_items/106_
zipper-a/deleted_items/352_
zipper-a/deleted_items/500_
zipper-a/deleted_items/397_
zipper-a/deleted_items/408_
zipper-a/deleted_items/249_
zipper-a/deleted_items/333_
zipper-a/deleted_items/454_
zipper-a/deleted_items/276_
zipper-a/deleted_items/230_
zipper-a/deleted_items/391_
zipper-a/deleted_items/372_
zipper-a/deleted_items/112_
zipper-a/deleted_items/331_
zipper-a/deleted_items/151_
zipper-a/deleted_items/233_
zipper-a/deleted_items

zipper-a/sent_items/47_
zipper-a/sent_items/155_
zipper-a/sent_items/54_
zipper-a/sent_items/17_
zipper-a/sent_items/269_
zipper-a/sent_items/265_
zipper-a/sent_items/130_
zipper-a/sent_items/374_
zipper-a/sent_items/139_
zipper-a/sent_items/196_
zipper-a/sent_items/103_
zipper-a/sent_items/264_
zipper-a/sent_items/160_
zipper-a/sent_items/303_
zipper-a/sent_items/40_
zipper-a/sent_items/280_
zipper-a/sent_items/376_
zipper-a/sent_items/170_
zipper-a/sent_items/117_
zipper-a/sent_items/171_
zipper-a/sent_items/366_
zipper-a/sent_items/11_
zipper-a/sent_items/91_
zipper-a/sent_items/32_
zipper-a/sent_items/337_
zipper-a/sent_items/214_
zipper-a/sent_items/353_
zipper-a/sent_items/33_
zipper-a/sent_items/245_
zipper-a/sent_items/2_
zipper-a/sent_items/255_
zipper-a/sent_items/338_
zipper-a/sent_items/292_
zipper-a/sent_items/92_
zipper-a/sent_items/355_
zipper-a/sent_items/102_
zipper-a/sent_items/132_
zipper-a/sent_items/70_
zipper-a/sent_items/30_
zipper-a/sent_items/239_
zipper-a/sent

shively-h/deleted_items/15_
shively-h/deleted_items/606_
shively-h/deleted_items/513_
shively-h/deleted_items/7_
shively-h/deleted_items/36_
shively-h/deleted_items/425_
shively-h/deleted_items/491_
shively-h/deleted_items/498_
shively-h/deleted_items/665_
shively-h/deleted_items/89_
shively-h/deleted_items/547_
shively-h/deleted_items/545_
shively-h/deleted_items/47_
shively-h/deleted_items/464_
shively-h/deleted_items/155_
shively-h/deleted_items/54_
shively-h/deleted_items/585_
shively-h/deleted_items/514_
shively-h/deleted_items/662_
shively-h/deleted_items/428_
shively-h/deleted_items/130_
shively-h/deleted_items/712_
shively-h/deleted_items/592_
shively-h/deleted_items/581_
shively-h/deleted_items/374_
shively-h/deleted_items/460_
shively-h/deleted_items/426_
shively-h/deleted_items/139_
shively-h/deleted_items/196_
shively-h/deleted_items/394_
shively-h/deleted_items/706_
shively-h/deleted_items/103_
shively-h/deleted_items/579_
shively-h/deleted_items/699_
shively-h/deleted_ite

shively-h/all_documents/195_
shively-h/all_documents/193_
shively-h/all_documents/22_
shively-h/all_documents/266_
shively-h/all_documents/204_
shively-h/all_documents/46_
shively-h/all_documents/137_
shively-h/all_documents/157_
shively-h/all_documents/215_
shively-h/all_documents/144_
shively-h/all_documents/52_
shively-h/all_documents/175_
shively-h/all_documents/162_
shively-h/all_documents/135_
shively-h/all_documents/273_
shively-h/all_documents/3_
shively-h/all_documents/28_
shively-h/all_documents/192_
shively-h/all_documents/109_
shively-h/all_documents/178_
shively-h/all_documents/115_
shively-h/all_documents/223_
shively-h/all_documents/136_
shively-h/all_documents/229_
shively-h/all_documents/38_
shively-h/all_documents/268_
shively-h/all_documents/19_
shively-h/all_documents/253_
shively-h/all_documents/42_
shively-h/all_documents/279_
shively-h/all_documents/293_
shively-h/all_documents/84_
shively-h/all_documents/290_
shively-h/all_documents/222_
shively-h/all_documents/

shively-h/inbox/120_
shively-h/inbox/99_
shively-h/inbox/57_
shively-h/inbox/95_
shively-h/inbox/8_
shively-h/inbox/63_
shively-h/inbox/59_
shively-h/inbox/27_
shively-h/inbox/53_
shively-h/inbox/124_
shively-h/inbox/69_
shively-h/inbox/64_
shively-h/inbox/62_
shively-h/inbox/93_
shively-h/inbox/66_
shively-h/inbox/71_
shively-h/inbox/37_
shively-h/inbox/133_
shively-h/inbox/125_
shively-h/inbox/41_
shively-h/inbox/126_
shively-h/inbox/65_
shively-h/inbox/4_
shively-h/inbox/49_
shively-h/inbox/88_
shively-h/inbox/72_
shively-h/inbox/83_
shively-h/inbox/75_
shively-h/inbox/22_
shively-h/inbox/46_
shively-h/inbox/137_
shively-h/inbox/52_
shively-h/inbox/175_
shively-h/inbox/135_
shively-h/inbox/3_
shively-h/inbox/28_
shively-h/inbox/109_
shively-h/inbox/178_
shively-h/inbox/115_
shively-h/inbox/136_
shively-h/inbox/104_
shively-h/inbox/38_
shively-h/inbox/19_
shively-h/inbox/82_
shively-h/inbox/42_
shively-h/inbox/84_
shively-h/inbox/73_
shively-h/inbox/110_
shively-h/inbox/127_
shively-

In [67]:
df.show()

+--------------------+--------+--------------------+
|                  id|username|        original_msg|
+--------------------+--------+--------------------+
|      gay-r/to_do/1_|   gay-r|Message-ID: <2414...|
|     gay-r/to_do/12_|   gay-r|Message-ID: <2300...|
|      gay-r/to_do/6_|   gay-r|Message-ID: <2677...|
|      gay-r/to_do/5_|   gay-r|Message-ID: <5586...|
|      gay-r/to_do/9_|   gay-r|Message-ID: <2400...|
|     gay-r/to_do/13_|   gay-r|Message-ID: <1780...|
|      gay-r/to_do/7_|   gay-r|Message-ID: <1399...|
|     gay-r/to_do/11_|   gay-r|Message-ID: <1311...|
|      gay-r/to_do/2_|   gay-r|Message-ID: <2330...|
|      gay-r/to_do/4_|   gay-r|Message-ID: <2590...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <2495...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <2526...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <1070...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <2709...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <2319...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <24

In [18]:
df.printSchema()

root
 |-- id_path: string (nullable = true)
 |-- username: string (nullable = true)
 |-- original_msg: string (nullable = true)



## Assignment 4.2

Use `plain_msg_example` and `html_msg_example` to create a function that parses an email message. 

In [47]:
plain_msg_example = """
Message-ID: <6742786.1075845426893.JavaMail.evans@thyme>
Date: Thu, 7 Jun 2001 11:05:33 -0700 (PDT)
From: jeffrey.hammad@enron.com
To: andy.zipper@enron.com
Subject: Thanks for the interview
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Hammad, Jeffrey </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=CBBE377A-24F58854-862567DD-591AE7>
X-To: Zipper, Andy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=AZIPPER>
X-cc: 
X-bcc: 
X-Folder: \Zipper, Andy\Zipper, Andy\Inbox
X-Origin: ZIPPER-A
X-FileName: Zipper, Andy.pst

Andy,

Thanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  

Thanks and Best Regards,

Jeff Hammad
"""

html_msg_example = """
Message-ID: <21013632.1075862392611.JavaMail.evans@thyme>
Date: Mon, 19 Nov 2001 12:15:44 -0800 (PST)
From: insynconline.6jy5ympb.d@insync-palm.com
To: tstaab@enron.com
Subject: Last chance for special offer on Palm OS Upgrade!
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: InSync Online <InSyncOnline.6jy5ympb.d@insync-palm.com>
X-To: THERESA STAAB <tstaab@enron.com>
X-cc: 
X-bcc: 
X-Folder: \TSTAAB (Non-Privileged)\Staab, Theresa\Deleted Items
X-Origin: Staab-T
X-FileName: TSTAAB (Non-Privileged).pst

<html>

<html>
<head>
<title>Paprika</title>
<meta http-equiv="Content-Type" content="text/html;">
</head>
<body bgcolor="#FFFFFF" TEXT="#333333" LINK="#336699" VLINK="#6699cc" ALINK="#ff9900">
<table border="0" cellpadding="0" cellspacing="0" width="582">
<tr valign="top">
  <td width="582" colspan="9"><nobr><a href="http://insync-online.p04.com/u.d?BEReaQA5eczXB=1"><img src="http://images4.postdirect.com/master-images/404707/upper_left.gif" alt="" width="103" height="110" hspace="0" vspace="0" border="0"></a><a href="http://insync-online.p04.com/u.d?AkReaQA5eczXE=11"><img src="http://images4.postdirect.com/master-images/404707/upper_right.gif" alt="" width="479" height="110" hspace="0" vspace="0" border="0"></a></nobr></td>
</tr>
<tr valign="top">
  <td width="4" bgcolor="#CCCCCC"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="4" height="1" hspace="0" vspace="0" border="0" alt=""></td>
  <td width="20"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="20" height="1" hspace="0" vspace="0" border="0" alt=""></td>
  <td width="165"><br><a href="http://insync-online.p04.com/u.d?LkReaQA5eczXL=21"><img src="http://images4.postdirect.com/master-images/404707/screen1.gif" alt="" width="165" height="159" hspace="0" vspace="0" border="0"></a><br><img src="http://images4.postdirect.com/master-images/404707/screen1_text.gif" alt="" width="93" height="26" hspace="0" vspace="0" border="0"></td>
  <td width="20"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="20" height="1" hspace="0" vspace="0" border="0" alt=""></td>
  <td width="165"><br><a href="http://insync-online.p04.com/u.d?BkReaQA5eczXO=31"><img src="http://images4.postdirect.com/master-images/404707/screen2.gif" alt="" width="165" height="159" hspace="0" vspace="0" border="0"></a><br><img src="http://images4.postdirect.com/master-images/404707/screen2_text.gif" alt="" width="93" height="26" hspace="0" vspace="0" border="0"></td>
  <td width="20"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="20" height="1" hspace="0" vspace="0" border="0" alt=""></td>
  <td width="165"><br><a href="http://insync-online.p04.com/u.d?JkReaQA5eczXRs=41"><img src="http://images4.postdirect.com/master-images/404707/screen3.gif" alt="" width="165" height="159" hspace="0" vspace="0" border="0"></a><br><img src="http://images4.postdirect.com/master-images/404707/screen3_text.gif" alt="" width="93" height="26" hspace="0" vspace="0" border="0"></td>
  <td width="19"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="19" height="1" hspace="0" vspace="0" border="0" alt=""></td>
  <td width="4" bgcolor="#CCCCCC"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="4" height="1" hspace="0" vspace="0" border="0" alt=""></td>
</tr>
</table>
<table border="0" cellpadding="0" cellspacing="0" width="582">
<tr valign="top">
  <td width="4" bgcolor="#CCCCCC"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="4" height="1" hspace="0" vspace="0" border="0" alt=""></td>
  <td width="574"><br>
    <table border="0" cellpadding="0" cellspacing="0" width="574" bgcolor="#99ccff">
    <tr>
      <td width="50"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="50" height="1" hspace="0" vspace="0" border="0" alt=""></td>
      <td width="474"><font face="verdana, arial" size="-2"color="#000000">
        <br>
        Dear THERESA,
        <br><br>
        Due to overwhelming demand for the Palm OS&#174; v4.1 Upgrade with Mobile Connectivity, we are 
        extending the special offer of 25% off through November 30, 2001. So there's still time to significantly 
        increase the functionality of your Palm&#153; III, IIIx, IIIxe, IIIc, V or Vx handheld. Step up to the 
        new Palm OS v4.1 through this extended special offer. You'll receive the brand new Palm OS v4.1 
        <b>for just $29.95 when you use Promo Code <font color="#FF0000">OS41WAVE</font></b>. That's a 
        <b>$10 savings</b> off the list price. 
        <br><br>
        <a href="http://insync-online.p04.com/u.d?NkReaQA5eczXRh=51">Click here to view a full product demo now</a>.
        <br><br>
        <a href="http://insync-online.p04.com/u.d?MkReaQA5eczXRm=61"><img src="http://images4.postdirect.com/master-images/404707/title1.gif" alt="" width="336" height="20" hspace="0" vspace="0" border="0"></a>
        <br><br>
        You can do a lot more with your Palm&#153; handheld when you upgrade to the Palm OS v4.1. All your 
        favorite features just got even better and there are some terrific new additions:
        <br><br>
        <LI> Handwrite notes and even draw pictures right on your Palm&#153 handheld</LI>
        <LI> Tap letters with your stylus and use Graffiti&#174; at the same time with the enhanced onscreen keyboard</LI>
        <LI> Improved Date Book functionality lets you view, snooze or clear multiple alarms all with a single tap </LI>
        <LI> You can easily change time-zone settings</LI>
        
        <br><br>
        <a href="http://insync-online.p04.com/u.d?WkReaQA5eczXRb=71"><img src="http://images4.postdirect.com/master-images/404707/title2.gif" alt="" width="460" height="20" hspace="0" vspace="0" border="0"></a>
        <br><br>
        <LI> <nobr>Mask/unmask</nobr> private records or hide/unhide directly within the application</LI>
        <LI> Lock your device automatically at a designated time using the new Autolocking feature</LI>
        <LI> Always remember your password with our new Hint feature*</LI>
        
        <br><br>
        <a href="http://insync-online.p04.com/u.d?VEReaQA5eczXRQ=81"><img src="http://images4.postdirect.com/master-images/404707/title3.gif" alt="" width="461" height="31" hspace="0" vspace="0" border="0"></a>
        <br><br>
        <LI> Use your GSM compatible mobile phone or modem to get online and access the web</LI>
        <LI> Stay connected with email, instant messaging and text messaging to GSM mobile phones</LI>
        <LI> Send applications or records through your cell phone to schedule meetings and even "beam" 
             important information to others</LI>
        
        <br><br>
        All this comes in a new operating system that can be yours for just $29.95! <a href="http://insync-online.p04.com/u.d?MkReaQA5eczXRV=91">Click here to 
        upgrade to the new Palm&#153; OS v4.1</a> and you'll also get the latest Palm desktop software. Or call 
        <nobr>1-800-881-7256</nobr> to order via phone. 
        <br><br>
        Sincerely,<br>
        The Palm Team
        <br><br>
        P.S. Remember, this extended offer opportunity of 25% savings absolutely ends on November 30, 2001 
        and is only available through the Palm Store when you use Promo Code <b><font color="#FF0000">OS41WAVE</font></b>.
        <br><br>
        <img src="http://images4.postdirect.com/master-images/404707/bottom_button.gif" align="right" alt="" width="295" height="60" hspace="0" vspace="0" border="0">
        <br><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="474" height="1" hspace="0" vspace="0" border="0" alt="">
        </font></td>
      <td width="50"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="50" height="1" hspace="0" vspace="0" border="0" alt=""></td>
    </tr>
    </table></td>
    <td width="4" bgcolor="#CCCCCC"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="4" height="1" hspace="0" vspace="0" border="0" alt=""></td>
  </tr>
  <tr>
  <td colspan="3"><img src="http://images4.postdirect.com/master-images/404707/bottom.gif" width="582" height="67" hspace="0" vspace="0" border="0"></td>
  </tr>
</table>
<table border="0" cellpadding="0" cellspacing="0" width="582">
  <tr>
    <td width="54"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="54" height="1" hspace="0" vspace="0" border="0" alt=""></td>
    <td width="474"><font face="arial, verdana" size="-2" color="#000000"><br>
    * This feature is available on the Palm&#153; IIIx, Palm&#153; IIIxe, and Palm&#153; Vx. <br><br>
    ** Note: To use the MIK functionality, you need either a Palm OS&#174; compatible modem or a phone 
    with  <nobr>built-in</nobr> modem or data capability that has either an infrared port or cable exits.  If you 
    are using a phone, you must have data services from your mobile service provider.  <a href="http://insync-online.p04.com/u.d?RkReaQA5eczXRK=101">Click here</a> for 
    a list of tested and supported phones that you can use with the MIK. Cable not provided.
    <br><br>
    ------------------<br>
    To modify your profile or unsubscribe from Palm newsletters, <a href="http://insync-online.p04.com/u.d?KkReaQA5eczXRE=121">click here</a>. 
    Or, unsubscribe by replying to this message, with "unsubscribe" as the subject line of the message. 
    <br><br>
    ------------------<br>
    Copyright&#169; 2001 Palm, Inc. Palm OS, Palm Computing, HandFAX, HandSTAMP, HandWEB, Graffiti, 
    HotSync, iMessenger, MultiMail, Palm.Net, PalmConnect, PalmGlove, PalmModem, PalmPoint, PalmPrint, 
    and the Palm Platform Compatible Logo are registered trademarks of Palm, Inc. Palm, the Palm logo, 
    AnyDay, EventClub, HandMAIL, the HotSync Logo, PalmGear, PalmGlove, PalmPix, Palm Powered, the Palm 
    trade dress, PalmSource, Smartcode, and Simply Palm are trademarks of Palm, Inc. All other brands and 
    product names may be trademarks or registered trademarks of their respective owners.</font>
    <img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="474" height="1" hspace="0" vspace="0" border="0" alt=""></td>
    <td width="54"><img src="http://images4.postdirect.com/master-images/404707/clear.gif" width="54" height="1" hspace="0" vspace="0" border="0" alt=""></td>
  </tr>
</table><br><br><br><br>
<!-- The following image is included for message detection -->
<img src="http://p04.com/1x1.dyn" border="0" alt="" width="1" height="1">
<img src="http://p04.com/1x1.dyn?0vEGou8Ig30ba2L2bLn" width=1 height=1></body>
</html>

</html>
"""
plain_msg_example = plain_msg_example.strip()
html_msg_example = html_msg_example.strip()

In [3]:
plain_msg_example

'Message-ID: <6742786.1075845426893.JavaMail.evans@thyme>\nDate: Thu, 7 Jun 2001 11:05:33 -0700 (PDT)\nFrom: jeffrey.hammad@enron.com\nTo: andy.zipper@enron.com\nSubject: Thanks for the interview\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Hammad, Jeffrey </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=CBBE377A-24F58854-862567DD-591AE7>\nX-To: Zipper, Andy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=AZIPPER>\nX-cc: \nX-bcc: \nX-Folder: \\Zipper, Andy\\Zipper, Andy\\Inbox\nX-Origin: ZIPPER-A\nX-FileName: Zipper, Andy.pst\n\nAndy,\n\nThanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  \n\nThanks and Best Regards,\n\nJeff Hammad'

In [4]:
print(plain_msg_example)

Message-ID: <6742786.1075845426893.JavaMail.evans@thyme>
Date: Thu, 7 Jun 2001 11:05:33 -0700 (PDT)
From: jeffrey.hammad@enron.com
To: andy.zipper@enron.com
Subject: Thanks for the interview
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Hammad, Jeffrey </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=CBBE377A-24F58854-862567DD-591AE7>
X-To: Zipper, Andy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=AZIPPER>
X-cc: 
X-bcc: 
X-Folder: \Zipper, Andy\Zipper, Andy\Inbox
X-Origin: ZIPPER-A
X-FileName: Zipper, Andy.pst

Andy,

Thanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  

Thanks and Best Regards,

Jeff Hammad


In [15]:
html_msg_example[:1000]

'Message-ID: <21013632.1075862392611.JavaMail.evans@thyme>\nDate: Mon, 19 Nov 2001 12:15:44 -0800 (PST)\nFrom: insynconline.6jy5ympb.d@insync-palm.com\nTo: tstaab@enron.com\nSubject: Last chance for special offer on Palm OS Upgrade!\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: InSync Online <InSyncOnline.6jy5ympb.d@insync-palm.com>\nX-To: THERESA STAAB <tstaab@enron.com>\nX-cc: \nX-bcc: \nX-Folder: \\TSTAAB (Non-Privileged)\\Staab, Theresa\\Deleted Items\nX-Origin: Staab-T\nX-FileName: TSTAAB (Non-Privileged).pst\n\n<html>\n\n<html>\n<head>\n<title>Paprika</title>\n<meta http-equiv="Content-Type" content="text/html;">\n</head>\n<body bgcolor="#FFFFFF" TEXT="#333333" LINK="#336699" VLINK="#6699cc" ALINK="#ff9900">\n<table border="0" cellpadding="0" cellspacing="0" width="582">\n<tr valign="top">\n  <td width="582" colspan="9"><nobr><a href="http://insync-online.p04.com/u.d?BEReaQA5eczXB=1"><img src="http://images4.postdirect.co

In [16]:
print(html_msg_example)

Message-ID: <21013632.1075862392611.JavaMail.evans@thyme>
Date: Mon, 19 Nov 2001 12:15:44 -0800 (PST)
From: insynconline.6jy5ympb.d@insync-palm.com
To: tstaab@enron.com
Subject: Last chance for special offer on Palm OS Upgrade!
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: InSync Online <InSyncOnline.6jy5ympb.d@insync-palm.com>
X-To: THERESA STAAB <tstaab@enron.com>
X-cc: 
X-bcc: 
X-Folder: \TSTAAB (Non-Privileged)\Staab, Theresa\Deleted Items
X-Origin: Staab-T
X-FileName: TSTAAB (Non-Privileged).pst

<html>

<html>
<head>
<title>Paprika</title>
<meta http-equiv="Content-Type" content="text/html;">
</head>
<body bgcolor="#FFFFFF" TEXT="#333333" LINK="#336699" VLINK="#6699cc" ALINK="#ff9900">
<table border="0" cellpadding="0" cellspacing="0" width="582">
<tr valign="top">
  <td width="582" colspan="9"><nobr><a href="http://insync-online.p04.com/u.d?BEReaQA5eczXB=1"><img src="http://images4.postdirect.com/master-images/404707/upper_l

Implement a function that  
1) takes the **path to an email file** and  
2) returns a **dictionary**  
3) **containing** the fields listed in the previous table:  
* username (name of email folder)  
* original_msg (unparsed email)  
* payload (unparsed payload)  
* From email header:  
 * Message-ID  
 * Date (parsed datetime)  
 * From  
 * To  
 * Subject  
 * Mime-version  
 * Content-Type  
 * Content-Transfer-Encoding  
 * X-From  
 * X-To  
 * X-cc  
 * X-bcc  
 * X-Folder  
 * X-Origin  
 * X-FileName  
 * Cc (?)  
 * Bcc (?)  

In [5]:
for i in headers:
    print(i)

NameError: name 'headers' is not defined

In [8]:
def parse_html_payload(payload):
    """
    This function uses Beautiful Soup to read HTML data
    and return the text.  If the payload is plain text, then
    Beautiful Soup will return the original content
    """
    soup = BeautifulSoup(payload, 'html.parser')
    return str(soup.get_text()).encode('utf-8').decode('utf-8')

In [ ]:
def parse_email(original_msg):
    result = {}
    msg = Parser(policy=default).parsestr(original_msg)
    ## TODO: Use Python's email library to read the payload and the headers
    ## https://docs.python.org/3/library/email.examples.html
    tuple_result = tuple([str(result.get(column, None)) for column in columns])
    return ParsedEmail(*tuple_result)

plain_msg_example; html_msg_example

In [9]:
# from email.parser import BytesParser
from email.parser import Parser

In [10]:
headers = Parser(policy=default).parsestr(plain_msg_example)

In [11]:
headers

In [12]:
for i in headers:
    print(i)

Message-ID
Date
From
To
Subject
Mime-Version
Content-Type
Content-Transfer-Encoding
X-From
X-To
X-cc
X-bcc
X-Folder
X-Origin
X-FileName


In [13]:
headers['To']

'andy.zipper@enron.com'

In [14]:
for i in  headers:
    print(f'{i}:')
    print(f' {headers[i]}\n')

Message-ID:
 <6742786.1075845426893.JavaMail.evans@thyme>

Date:
 Thu, 07 Jun 2001 11:05:33 -0700

From:
 jeffrey.hammad@enron.com

To:
 andy.zipper@enron.com

Subject:
 Thanks for the interview

Mime-Version:
 1.0

Content-Type:
 text/plain; charset="us-ascii"

Content-Transfer-Encoding:
 7bit

X-From:
 Hammad, Jeffrey </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=CBBE377A-24F58854-862567DD-591AE7>

X-To:
 Zipper, Andy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=AZIPPER>

X-cc:
 

X-bcc:
 

X-Folder:
 \Zipper, Andy\Zipper, Andy\Inbox

X-Origin:
 ZIPPER-A

X-FileName:
 Zipper, Andy.pst



In [29]:
print(plain_msg_example)

Message-ID: <6742786.1075845426893.JavaMail.evans@thyme>
Date: Thu, 7 Jun 2001 11:05:33 -0700 (PDT)
From: jeffrey.hammad@enron.com
To: andy.zipper@enron.com
Subject: Thanks for the interview
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Hammad, Jeffrey </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=CBBE377A-24F58854-862567DD-591AE7>
X-To: Zipper, Andy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=AZIPPER>
X-cc: 
X-bcc: 
X-Folder: \Zipper, Andy\Zipper, Andy\Inbox
X-Origin: ZIPPER-A
X-FileName: Zipper, Andy.pst

Andy,

Thanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  

Thanks and Best Regards,

Jeff Hammad


In [16]:
msg = Parser().parsestr(plain_msg_example)

In [23]:
msg.get_content_maintype()

'text'

In [24]:
for part in msg.walk():
    if part.get_content_maintype() == 'multipart':
        continue
    filename = part.get_filename()
    if not filename:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-24-ec2e09993c3a>, line 6)

In [25]:
msg.get_filename()

In [26]:
import mimetypes
ext = mimetypes.guess_extension(msg.get_content_type())

In [27]:
ext

'.txt'

In [ ]:
def parse_email(original_msg):
    result = {}
    columns = ['username', 'original_msg', 'payload', 'Message-ID', 'Date', 'From', 'To', 
               'Subject', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-From', 
               'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Cc', 'Bcc']
    msg = Parser(policy=default).parsestr(original_msg)
    ## TODO: Use Python's email library to read the payload and the headers
    ## https://docs.python.org/3/library/email.examples.html
    tuple_result = tuple([str(result.get(column, None)) for column in columns])
    return ParsedEmail(*tuple_result)

In [49]:
result = {}

In [29]:
msg = Parser(policy=default).parsestr(plain_msg_example)

In [43]:
# columns = ['username', 'original_msg', 'payload']

In [44]:
# for i in msg:
#     columns.append(i)

In [46]:
columns = ['username', 'original_msg', 'payload', 'Message-ID', 'Date', 'From', 'To', 
           'Subject', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-From', 
           'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Cc', 'Bcc']

In [33]:
msg.get_payload()

'Andy,\n\nThanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  \n\nThanks and Best Regards,\n\nJeff Hammad'

In [42]:
type(msg)

email.message.EmailMessage

In [51]:
result['username'] = 'testUsername'

In [53]:
result['original_msg'] = plain_msg_example

In [55]:
result['payload'] = msg.get_payload()

In [61]:
for i in msg:
#     print(i)
#     print(msg[i])
    result[i] = msg[i]

In [63]:
# result

In [75]:
[str(result.get(column, None)) for column in columns]

['testUsername',
 'Message-ID: <6742786.1075845426893.JavaMail.evans@thyme>\nDate: Thu, 7 Jun 2001 11:05:33 -0700 (PDT)\nFrom: jeffrey.hammad@enron.com\nTo: andy.zipper@enron.com\nSubject: Thanks for the interview\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Hammad, Jeffrey </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NOTESADDR/CN=CBBE377A-24F58854-862567DD-591AE7>\nX-To: Zipper, Andy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=AZIPPER>\nX-cc: \nX-bcc: \nX-Folder: \\Zipper, Andy\\Zipper, Andy\\Inbox\nX-Origin: ZIPPER-A\nX-FileName: Zipper, Andy.pst\n\nAndy,\n\nThanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  \n\nThanks and Best Regards,\n\nJeff Hammad',
 'Andy,\n\nThanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to co

In [76]:
len(tuple_result)

20

In [64]:
tuple_result = tuple([str(result.get(column, None)) for column in columns])

In [85]:
def parse_email(original_msg):
    result = {}
#     columns = ['username', 'original_msg', 'payload', 'Message-ID', 'Date', 'From', 'To', 
#                'Subject', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-From', 
#                'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Cc', 'Bcc']
    msg = Parser(policy=default).parsestr(original_msg)
    
    ## TODO: Use Python's email library to read the payload and the headers
    ## https://docs.python.org/3/library/email.examples.html
#     result['username'] = 'testUsername'
#     result['original_msg'] = original_msg
#     result['payload'] = msg.get_payload()
    result['text'] = msg.get_payload()
    
    for item in msg:
        result[item] = msg[item]
        
    tuple_result = tuple([str(result.get(column, None)) for column in columns])
    return ParsedEmail(*tuple_result)

In [86]:
parsed_msg = parse_email(plain_msg_example)

In [87]:
print(parsed_msg.text)

Andy,

Thanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  

Thanks and Best Regards,

Jeff Hammad


In [90]:
# print(html_msg_example)

In [92]:
parsedHTML = parse_html_payload(html_msg_example)

In [94]:
# parsedHTML_msg = parse_email(parsedHTML)

In [96]:
# print(parsedHTML_msg.text)

In [107]:
msg = Parser(policy=default).parsestr(html_msg_example)

In [108]:
msg

In [109]:
for i in msg:
    print(i)

Message-ID
Date
From
To
Subject
Mime-Version
Content-Type
Content-Transfer-Encoding
X-From
X-To
X-cc
X-bcc
X-Folder
X-Origin
X-FileName


In [110]:
msg.is_multipart()

False

In [111]:
payload = msg.get_payload()

In [120]:
# HTML_parsed = parse_html_payload(payload)
# HTML_parsed

In [121]:
html_msg = parse_html_payload(html_msg_example)

In [123]:
msg = Parser(policy=default).parsestr(html_msg)

In [128]:
payload = msg.get_payload()

In [130]:
print(payload)




Paprika




























        Dear THERESA,
        
        Due to overwhelming demand for the Palm OS® v4.1 Upgrade with Mobile Connectivity, we are 
        extending the special offer of 25% off through November 30, 2001. So there's still time to significantly 
        increase the functionality of your Palm™ III, IIIx, IIIxe, IIIc, V or Vx handheld. Step up to the 
        new Palm OS v4.1 through this extended special offer. You'll receive the brand new Palm OS v4.1 
        for just $29.95 when you use Promo Code OS41WAVE. That's a 
        $10 savings off the list price. 
        
Click here to view a full product demo now.
        


        You can do a lot more with your Palm™ handheld when you upgrade to the Palm OS v4.1. All your 
        favorite features just got even better and there are some terrific new additions:
        
 Handwrite notes and even draw pictures right on your Palm™ handheld
 Tap letters with your stylus and use Graffiti® at the same t

In [134]:
simplest = msg.get_body(preferencelist=('plain', 'html'))

In [138]:
print(simplest.get_content())




Paprika




























        Dear THERESA,
        
        Due to overwhelming demand for the Palm OS� v4.1 Upgrade with Mobile Connectivity, we are 
        extending the special offer of 25% off through November 30, 2001. So there's still time to significantly 
        increase the functionality of your Palm\u2122 III, IIIx, IIIxe, IIIc, V or Vx handheld. Step up to the 
        new Palm OS v4.1 through this extended special offer. You'll receive the brand new Palm OS v4.1 
        for just $29.95 when you use Promo Code OS41WAVE. That's a 
        $10 savings off the list price. 
        
Click here to view a full product demo now.
        


        You can do a lot more with your Palm\u2122 handheld when you upgrade to the Palm OS v4.1. All your 
        favorite features just got even better and there are some terrific new additions:
        
 Handwrite notes and even draw pictures right on your Palm\u2122 handheld
 Tap letters with your stylus and use Graffiti

In [140]:
print(''.join(simplest.get_content().splitlines(keepends=True)[:3]))

In [144]:
msg.get_body(preferencelist=( 'html'))

In [149]:
for line in msg.get_content().splitlines():
    print(line)




Paprika




























        Dear THERESA,
        
        Due to overwhelming demand for the Palm OS� v4.1 Upgrade with Mobile Connectivity, we are 
        extending the special offer of 25% off through November 30, 2001. So there's still time to significantly 
        increase the functionality of your Palm\u2122 III, IIIx, IIIxe, IIIc, V or Vx handheld. Step up to the 
        new Palm OS v4.1 through this extended special offer. You'll receive the brand new Palm OS v4.1 
        for just $29.95 when you use Promo Code OS41WAVE. That's a 
        $10 savings off the list price. 
        
Click here to view a full product demo now.
        


        You can do a lot more with your Palm\u2122 handheld when you upgrade to the Palm OS v4.1. All your 
        favorite features just got even better and there are some terrific new additions:
        
 Handwrite notes and even draw pictures right on your Palm\u2122 handheld
 Tap letters with your stylus and use Graffiti

In [151]:
msg = Parser(policy=default).parsestr(plain_msg_example)

In [152]:
parse_html_payload(msg.get_payload())

'Andy,\n\nThanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  \n\nThanks and Best Regards,\n\nJeff Hammad'

In [153]:
def parse_email(original_msg):
    result = {}

    msg = Parser(policy=default).parsestr(original_msg)
    
    ## TODO: Use Python's email library to read the payload and the headers
    ## https://docs.python.org/3/library/email.examples.html

    payload = msg.get_payload()

    result['text'] = parse_html_payload(payload)
    
    for item in msg:
        result[item] = msg[item]
        
    tuple_result = tuple([str(result.get(column, None)) for column in columns])
    return ParsedEmail(*tuple_result)

In [154]:
parsed_msg = parse_email(plain_msg_example)

In [155]:
print(parsed_msg.text)

Andy,

Thanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  

Thanks and Best Regards,

Jeff Hammad


In [156]:
parsed_html_msg = parse_email(html_msg_example)

In [157]:
print(parsed_html_msg.text)




Paprika




























        Dear THERESA,
        
        Due to overwhelming demand for the Palm OS® v4.1 Upgrade with Mobile Connectivity, we are 
        extending the special offer of 25% off through November 30, 2001. So there's still time to significantly 
        increase the functionality of your Palm™ III, IIIx, IIIxe, IIIc, V or Vx handheld. Step up to the 
        new Palm OS v4.1 through this extended special offer. You'll receive the brand new Palm OS v4.1 
        for just $29.95 when you use Promo Code OS41WAVE. That's a 
        $10 savings off the list price. 
        
Click here to view a full product demo now.
        


        You can do a lot more with your Palm™ handheld when you upgrade to the Palm OS v4.1. All your 
        favorite features just got even better and there are some terrific new additions:
        
 Handwrite notes and even draw pictures right on your Palm™ handheld
 Tap letters with your stylus and use Graffiti® at the same t

In [89]:
def parse_html_payload(payload):
    """
    This function uses Beautiful Soup to read HTML data
    and return the text.  If the payload is plain text, then
    Beautiful Soup will return the original content
    """
    soup = BeautifulSoup(payload, 'html.parser')
    return str(soup.get_text()).encode('utf-8').decode('utf-8')

def parse_email(original_msg):
    result = {}

    msg = Parser(policy=default).parsestr(original_msg)
    
    ## TODO: Use Python's email library to read the payload and the headers
    ## https://docs.python.org/3/library/email.examples.html

    payload = msg.get_payload()

    result['text'] = parse_html_payload(payload)
    
    for item in msg:
        result[item] = msg[item]
        
    tuple_result = tuple([str(result.get(column, None)) for column in columns])
    return ParsedEmail(*tuple_result)

In [90]:
parsed_msg = parse_email(plain_msg_example)

In [91]:
print(parsed_msg.text)

Andy,

Thanks for giving me the opportunity to meet with you about the Analyst/ Associate program.  I enjoyed talking to you, and look forward to contributing to the success that the program has enjoyed.  

Thanks and Best Regards,

Jeff Hammad


In [51]:
parsed_html_msg = parse_email(html_msg_example)

In [52]:
print(parsed_html_msg.text)




Paprika




























        Dear THERESA,
        
        Due to overwhelming demand for the Palm OS® v4.1 Upgrade with Mobile Connectivity, we are 
        extending the special offer of 25% off through November 30, 2001. So there's still time to significantly 
        increase the functionality of your Palm™ III, IIIx, IIIxe, IIIc, V or Vx handheld. Step up to the 
        new Palm OS v4.1 through this extended special offer. You'll receive the brand new Palm OS v4.1 
        for just $29.95 when you use Promo Code OS41WAVE. That's a 
        $10 savings off the list price. 
        
Click here to view a full product demo now.
        


        You can do a lot more with your Palm™ handheld when you upgrade to the Palm OS v4.1. All your 
        favorite features just got even better and there are some terrific new additions:
        
 Handwrite notes and even draw pictures right on your Palm™ handheld
 Tap letters with your stylus and use Graffiti® at the same t

## Assignment 4.3

Provided code:

In [2]:
## This creates a schema for the email data
email_struct = StructType()

for column in columns:
    email_struct.add(column, StringType(), True)

In [73]:
# print(email_struct)

In [ ]:
# ## This creates a user-defined function which can be used in Spark
# parse_email_func = udf(lambda z: parse_email(z), email_struct)

# def parse_emails(input_df):
#     new_df = input_df.select(
#         'username', 'id', 'original_msg', parse_email_func('original_msg').alias('parsed_email')
#     )
#     for column in columns:
#         new_df = new_df.withColumn(column, new_df.parsed_email[column])
    
#     new_df = new_df.drop('parsed_email')
#     return new_df
        
# class ParseEmailsTransformer(Transformer):
#     def _transform(self, dataset):
#         """
#         Transforms the input dataset.

#         :param dataset: input dataset, which is an instance of :py:class:`pyspark.sql.DataFrame`
#         :returns: transformed dataset
#         """
#         return dataset.transform(parse_emails)

# ## Use the custom ParseEmailsTransformer, Tokenizer, and CountVectorizer 
# ## to create a spark pipeline 
# email_pipeline = Pipeline(
#     ## TODO: Complete code
# )
# model = email_pipeline.fit(df)
# result = model.transform(df)

In [140]:
## This creates a user-defined function which can be used in Spark
parse_email_func = udf(lambda z: parse_email(z), email_struct)

In [141]:
def parse_emails(input_df):
    new_df = input_df.select(
        'username', 'id', 'original_msg', parse_email_func('original_msg').alias('parsed_email')
    )
    for column in columns:
        new_df = new_df.withColumn(column, new_df.parsed_email[column])
    
    new_df = new_df.drop('parsed_email')
    return new_df

In [142]:
class ParseEmailsTransformer(Transformer):
    def _transform(self, dataset):
        """
        Transforms the input dataset.

        :param dataset: input dataset, which is an instance of :py:class:`pyspark.sql.DataFrame`
        :returns: transformed dataset
        """
        return dataset.transform(parse_emails)

In [143]:
## Use the custom ParseEmailsTransformer, Tokenizer, and CountVectorizer 
## to create a spark pipeline 
tokenizer = Tokenizer(inputCol="text", outputCol="words")

cv = CountVectorizer(inputCol="words", outputCol="features")
# cv.setInputCol("words")
# cv.setOutputCol("features")

email_pipeline = Pipeline(stages=[ParseEmailsTransformer(), tokenizer, cv]
    ## TODO: Complete code
)


In [144]:
model = email_pipeline.fit(df)

In [145]:
result = model.transform(df)

In [104]:
# result

In [146]:
result.select('words').show()

+--------------------+
|               words|
+--------------------+
|[, task, assignme...|
|[, task, assignme...|
|[, task, assignme...|
|[, task, assignme...|
|[, task, assignme...|
|[, task, assignme...|
|[, task, assignme...|
|[, task, assignme...|
|[, task, assignme...|
|[, task, assignme...|
|[if, your, do, no...|
|[slow, browser, m...|
|[sean,, , attache...|
|[i, will, be, out...|
|[follow, olga, as...|
|[this, request, h...|
|[, everything, yo...|
|[, =09=09=09[imag...|
|[, 000, wtnt32, k...|
|[follow, olga, as...|
+--------------------+
only showing top 20 rows



In [147]:
result.select('id', 'words', 'features').show()

+--------------------+--------------------+--------------------+
|                  id|               words|            features|
+--------------------+--------------------+--------------------+
|      gay-r/to_do/1_|[, task, assignme...|(108640,[0,132,19...|
|     gay-r/to_do/12_|[, task, assignme...|(108640,[0,132,19...|
|      gay-r/to_do/6_|[, task, assignme...|(108640,[0,132,19...|
|      gay-r/to_do/5_|[, task, assignme...|(108640,[0,132,19...|
|      gay-r/to_do/9_|[, task, assignme...|(108640,[0,132,19...|
|     gay-r/to_do/13_|[, task, assignme...|(108640,[0,132,19...|
|      gay-r/to_do/7_|[, task, assignme...|(108640,[0,132,19...|
|     gay-r/to_do/11_|[, task, assignme...|(108640,[0,132,19...|
|      gay-r/to_do/2_|[, task, assignme...|(108640,[0,132,19...|
|      gay-r/to_do/4_|[, task, assignme...|(108640,[0,132,19...|
|gay-r/deleted_ite...|[if, your, do, no...|(108640,[0,1,2,3,...|
|gay-r/deleted_ite...|[slow, browser, m...|(108640,[0,1,2,3,...|
|gay-r/deleted_ite...|[se

In [88]:
df.show()

+--------------------+--------+--------------------+
|                  id|username|        original_msg|
+--------------------+--------+--------------------+
|      gay-r/to_do/1_|   gay-r|Message-ID: <2414...|
|     gay-r/to_do/12_|   gay-r|Message-ID: <2300...|
|      gay-r/to_do/6_|   gay-r|Message-ID: <2677...|
|      gay-r/to_do/5_|   gay-r|Message-ID: <5586...|
|      gay-r/to_do/9_|   gay-r|Message-ID: <2400...|
|     gay-r/to_do/13_|   gay-r|Message-ID: <1780...|
|      gay-r/to_do/7_|   gay-r|Message-ID: <1399...|
|     gay-r/to_do/11_|   gay-r|Message-ID: <1311...|
|      gay-r/to_do/2_|   gay-r|Message-ID: <2330...|
|      gay-r/to_do/4_|   gay-r|Message-ID: <2590...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <2495...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <2526...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <1070...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <2709...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <2319...|
|gay-r/deleted_ite...|   gay-r|Message-ID: <24

In [ ]:
result.select('id', 'words', 'features').show()

Playing around with directories:

In [41]:
enron_data_dir

PosixPath('/home/jovyan/dsc650/dsc650/assignments/assignment04/data/enron')

In [43]:
str(enron_data_dir).split('/', 5)

['',
 'home',
 'jovyan',
 'dsc650',
 'dsc650',
 'assignments/assignment04/data/enron']

In [44]:
str(enron_data_dir).split('/', 5)[-1]

'assignments/assignment04/data/enron'